In [1]:
# Initiate local llama3
! ollama pull llama3:latest

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 00e1317cbf74... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling ad1518640c43... 100% ▕████████████████▏  483 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling 00e1317cbf74... 100% ▕████████████████▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕████████████████▏  12 KB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                

In [2]:
! pip install --quiet langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python langchain-mistralai gpt4all

In [3]:
local_llm = 'llama3:latest'

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load
url = "https://lilianweng.github.io/posts/2023-06-23-agent/"
loader = WebBaseLoader(url)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=100
)
all_splits = text_splitter.split_documents(docs)

# Embed and index
embedding = GPT4AllEmbeddings()

# Index
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embedding,
)
retriever = vectorstore.as_retriever()

In [5]:
docs = retriever.get_relevant_documents('Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.')

/Users/liao/myds/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [6]:
docs[0].page_content

'Memory\n\nShort-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.\nLong-term memory: This provides the agent with the capability to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval.\n\n\nTool use\n\nThe agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.'

In [7]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.output_parsers import JsonOutputParser

## Use prompt from "langgraph/examples/rag/langgraph_crag_local.ipynb" in github

We could see for irrelevant question "This sentence doesn't exist in corpus" it still returns yes for relevance.
<br> And it won't return any supporting tokens despite being requested in prompt

In [8]:
# LLM

llm = ChatOllama(model=local_llm, format="json", temperature=0)


prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "This sentence doesn't exist in corpus"
# question = "How many types of memory in human brains"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[0].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}


In [9]:
prompt = PromptTemplate(
    template="""You are a grader assessing relevance of a retrieved document to a user question. \n 
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n
    If the document contains keywords related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and provide supporting tokens if there are relevant ones.""",
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()
# question = "This sentence doesn't exist in corpus"
question = "How many types of memory in human brains"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[0].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}


After modifying prompt we could see it correctly detected irrelevant sentence 
<br> and returns revelant tokens in retrievals

In [10]:

prompt = PromptTemplate(
    template="""
    
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n

    You are a assistant assessing relevance of a retrieved document to a user question.\n
    If the document contains corpus related to the user question, grade it as relevant.\n
    The goal is to filter out erroneous retrievals. \n
    Give the tokens which support your judgement and also include a binary score 'yes' or 'no' to indicate whether the document is relevant to the question.<|eot_id|>\n

    Here is the retrieved document: {document} <|eot_id|>\n
    Here is the user question: {question} <|eot_id|>\n
     
    <|start_header_id|>user<|end_header_id|>
    {question}<|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    
    """,
    input_variables=["question", "document"]
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "This sentence doesn't exist in corpus"
# question = "Generative Agents"
# question = "How many types of memory in human brains"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[0].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'no'}


In [11]:
prompt = PromptTemplate(
    template="""
    
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n

    You are a assistant assessing relevance of a retrieved document to a user question.\n
    If the document contains corpus related to the user question, grade it as relevant.\n
    The goal is to filter out erroneous retrievals. \n
    Give the tokens which support your judgement and also include a binary score 'yes' or 'no' to indicate whether the document is relevant to the question.<|eot_id|>\n

    Here is the retrieved document: {document} <|eot_id|>\n
    Here is the user question: {question} <|eot_id|>\n
     
    <|start_header_id|>user<|end_header_id|>
    {question}<|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>
    
    """,
    input_variables=["question", "document"]
)

retrieval_grader = prompt | llm | JsonOutputParser()
# question = "This sentence doesn't exist in corpus"
question = "How many types of memory in human brains"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[0].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'tokens': ['memory', 'human', 'brains'], 'score': 'yes'}
